<h1 style="text-align: center;" markdown="1">Complex Networks - Practical Session 2</h1>
<h3 style="text-align: center;" markdown="1">by Dimitri Lajou and Fabrice Lebeau</h3>

## Introduction
In this notebook, we implement and illustrates two methods of constructing random graphs: the **Erdös-Rényi random graph model** and the **configuration model of random networks**. 

Let us first import the tools we are going to use and our own `Graph` class that we began to define in the last practical session. 

**Important note:** in order for our examples to be more interactive, we are using `ipywidgets`. You may have to allow this extension for Jupyter by executing the following command in a terminal:
    
    jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [ ]:
import numpy as np
import math
import random
import scipy
import scipy.special
import matplotlib
import matplotlib.pyplot as plt;
%matplotlib inline  
plt.rcParams['figure.figsize'] = (15, 9)
plt.rcParams['font.size'] = 14
from IPython.display import Math, Markdown, Latex, display, display_latex, SVG
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
""" CR15 graph library """
class Graph(object):

    def __init__(self, graph_dict={}, graph_name=""):
        """ initializes a graph object """
        self.__graph_dict = graph_dict.copy()
        self.__name=graph_name

    def name(self):
        return self.__name
        
    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def connected_components(self):
        return self.__generate_components()

    def add_vertex(self, vertex):
        """ If vertex is not in self.__graph_dict, a key "vertex" with an empty
        list as a value is added to the dictionary. Otherwise nothing has to be 
        done."""
        if not vertex in self.__graph_dict:
            self.__graph_dict[vertex] = []
        

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list. No loops or 
        multiple edges."""
        my_edge = list(edge)
        if len(my_edge) != 2: raise WrongSizeForEdge()
        u = edge.pop()
        v = edge.pop()
        if u in self.__graph_dict and v in self.__graph_dict:
            if u != v:
                if v not in self.__graph_dict[u]:
                    self.__graph_dict[u].append(v)
                if u not in self.__graph_dict[v]:
                    self.__graph_dict[v].append(u)
        else:
            raise VerticesNotDecleared()
            

    def __generate_edges(self):
        """ A static method generating the edges of the graph "graph". Edges 
        are represented as sets two vertices, with no loops. To complete."""
        edges = []
        for v, edges_list in self.__graph_dict.items():
            for u in edges_list:
                if v < u:
                    edges.append(set([v,u]))
        return edges
    
    def vertex_degrees(self):
        """Return a dictionary degree"""
        degrees = {}
        for v, edges_list in self.__graph_dict.items():
            degrees[v] = len(edges_list)
        return degrees
    
    def vertex_degree(self, vertex):
        """Return a dictionary degree"""
        return len(self.__graph_dict[vertex])
    
    def find_isolated_vertices(self):
        """Return a set of zero-degree verticies"""
        zero_set = set()
        for v, edges_list in self.__graph_dict.items():
            if len(edges_list) == 0:
                zero_set.add(v)
        return zero_set
                
    def density(self):
        """Return the density of the graph"""
        deg = self.vertex_degrees()
        density = 0
        for v, d in deg.items():
            density += d
        density /=  (len(deg) -1) *len(deg)
        return density
                    
    def dict(self):
        return self.__graph_dict
    
    def degree_sequence(self):
        """Return the list of vertex degree sorted by decreasing degree"""
        deg = self.vertex_degrees()
        deg_list = [v for v in deg.values()]
        deg_list.sort(reverse=True)
        return tuple(deg_list)
    
    @staticmethod
    def erdos_gallai(deg_seq):
        """Given a degree sequence, this method verify that this sequence verify the erdos gallai conditions"""
        even_number = 0
        for v in deg_seq:
            even_number += v
        if even_number % 2 == 1 :
            return False
        sumOfdi = 0
        for k in range(len(deg_seq)):
            sumOfdi += deg_seq[k]
            sumOfMin = k*(k+1)
            for i in range(k, len(deg_seq)):
                sumOfMin += min(deg_seq[i], k+1)
            if sumOfMin < sumOfdi:
                return False
        return True
    
    def global_clustering_coefficient(self):
        triangle = 0
        triplet = 0
        for v in self.__graph_dict:
            for u in self.__graph_dict[v]:
                for w in self.__graph_dict[v]:
                    if u != w: 
                        triplet += 1
                    if u != w and w in self.__graph_dict[u]:
                        triangle += 1
        return triangle / triplet
    
    """ Graph traversal """
    def components_BFS(self, vertices, comps, base_vertice):
        if not vertices:
            return
        new_vertices = []
        for v in vertices:
            comps[v] = base_vertice
            for u in self.__graph_dict[v]:
                if comps[u] == u and u != base_vertice:
                    new_vertices.append(u)
        self.components_BFS(new_vertices, comps, base_vertice)
    
    def __generate_components(self):
        """Retuen a dictionnary of components representant"""
        comps = {}
        for u in self.__graph_dict:
            comps[u] = u
        for u in self.__graph_dict:
            if comps[u] == u:
                self.components_BFS([u], comps, u)
        return comps
    
    def shortest_path_BFS(self, vertices, seen, d, goal):
        if not vertices:
            return math.inf
        if goal in vertices:
            return d
        new_vertices = set()
        for v in vertices:
            seen.add(v)
        for v in vertices:
            for u in self.__graph_dict[v]:
                if u not in seen:
                    new_vertices.add(u)
        return self.shortest_path_BFS(new_vertices, seen, d+1, goal)
    
    def shortest_path(self, s, t):
        """Return the shortest path between s and t"""
        return self.shortest_path_BFS({s}, set(), 0, t)
    
    def diameter_BFS(self, vertices, seen, d):
        if not vertices:
            if d > 0:
                return d-1
            else:
                return d
        new_vertices = set()
        for v in vertices:
            seen.add(v)
        for v in vertices:
            for u in self.__graph_dict[v]:
                if u not in seen:
                    new_vertices.add(u)
        return self.diameter_BFS(new_vertices, seen, d+1)
    
    def diameter(self):
        diam = 0
        for u in self.__graph_dict:
            m = self.diameter_BFS({u}, set(), 0)
            if (m > diam):
                diam = m
        return diam
    
    def diameter_component(self, u):
        """ Return the diameter of the component containing vertex u """
        component = set()
        comps = self.connected_components()
        # First get the nodes of the component #
        for v in self.__graph_dict:
            if comps[v] == comps[u]:
                component.add(v)
        diam = 0
        for v in component:
            m = self.diameter_BFS({v}, set(), 0)
            if (m > diam):
                diam = m
        return diam
    
    def biggest_component_diameter(self):
        if not self.vertices(): return 0
        
        # First determine the biggest component #
        comps = self.connected_components()
        comps_size = {}
        for u in comps.values():
            comps_size[u] = 0
        for v,u in comps.items():
            comps_size[u] += 1
        biggest = tuple(comps.values())[0]
        max_size = comps_size[biggest]
        for u in comps.values():
            if comps_size[u] > max_size:
                max_size = comps_size[u]
                biggest = u
        
        # Return the diameter of the corresponding component
        return self.diameter_component(biggest)
        
    def spanning_tree(self):
        queue = []
        tree = []
        # Hack to get one key in the dict
        for v in self.__graph_dict:
            queue.append(v)
            break
        seen = [queue[-1]]

        while queue:
            u = queue.pop()
            for v in self.__graph_dict[u]:
                if not v in seen:
                    tree.append(set([u,v]))
                    seen.append(v)
                    queue.append(v)
        return tree
    
    def irregular_edge_count(self):
        nb_loop = 0
        nb_multi = 0
        nb_edge = 0
        for v in self.__graph_dict:
            seen = []
            for u in self.__graph_dict[v]:
                nb_edge += 1
                if u == v:
                    nb_loop += 1
                elif u in seen:
                    nb_multi += 1
                else:
                    seen.append(u)
        return (nb_loop + nb_multi // 2) / nb_edge * 2
        
    """ Static methods for defining classical graphs """
    @staticmethod
    def clique(n):
        d = {}
        s = set(i+1 for i in range(n))
        for i in range(n):
            d[i+1] = s.difference(set({i+1}))
        return Graph(d,"$K_"+str(n)+"$")
    
    @staticmethod
    def no_edges(n):
        d = {}
        for i in range(n):
            d[i+1] = []
        return Graph(d,"$D_"+str(n)+"$")

    """ Importing from a text file """
    @staticmethod
    def from_txt(file):
        G = Graph()
        lines = open(file).readlines()
        for l in lines:
            p = parse('{:d}\t{:d}', l)
            G.add_vertex(p[0])
            G.add_vertex(p[1])
            G.add_edge({p[0],p[1]})
        return G
    def print(self):
        print(self.__graph_dict)

## The Erdös-Rényi random graph model
### Generating ER graphs
We first implement the two variants that we are studying in this section:
- the first one, in function `er_np`, returns a graph (drawn from set $G_{n,p}$) with the specified number of nodes $n$ and each couple of nodes is linked independently from the others with the given probability $p$;
- the second one, in function `er_nm` returns a graph (drawn from set $G_{n,m}$) with the specified number of nodes $n$ and the exact given number of edges $m$ that are chosen uniformly among all possible edges.

In [ ]:
def er_np(n, p):
    graph_dict = {}
    for i in range(n):
        graph_dict[i] = []
    for i in range(n):
        for j in range(i):
            rv = random.uniform(0,1)
            if rv < p:
                graph_dict[i].append(j)
                graph_dict[j].append(i)  
    return Graph(graph_dict)

def er_nm(n, m):
    graph_dict = {}
    for i in range(n):
        graph_dict[i] = []
    # number of possible edges (*2 since it is easier to sample couple than pairs)
    nb_possibility = n*n
    count_left = m
    if m > n*(n-1)/2 :
        count_left = n*(n-1)/2
    while count_left > 0:
        # sample an index for the new edge
        edge_ind = random.randrange(nb_possibility)
        # get the two endpoints (part where it is easier for couple)
        i = edge_ind // n
        j = edge_ind % n
        # handle loops and multiedges
        if i == j :
            continue 
        if j in graph_dict[i]:
            continue
        graph_dict[i].append(j)
        graph_dict[j].append(i)
        count_left -= 1
    return Graph(graph_dict)

### Number of edges
A graph drawn from set $G_{n,p}$ has an expected number of edges which is $$\left\lfloor p \binom{n}{2} \right\rfloor.$$ We want to compare the number of edges obtained from graphs generated by our function `er_np` with the expected number of edges.

In [ ]:
def expected_edge_number(n, p):
    return math.floor(p * n*(n-1)/2)

def binomial_coef(n, k):
    return scipy.special.binom(n, k)

def compare_edge_count(n, p):
    """Generate an er_np graph and compare its edges to an er_nm"""
    m = expected_edge_number(n, p)
    G_tmp = er_np(n, p)
    nb_edges = len(G_tmp.edges())
    if m == 0:
        m = 1
    return nb_edges / m

def compare_edge_counts(n):
    x = [ p for p in np.arange(1/n, 1.0, 1/n)]
    got = [ compare_edge_count(n, p) for p in  np.arange(1/n, 1.0, 1/n)]
    plt.plot(x, got, label='Ratio |E| / $\\left\\lfloor p \\binom{n}{2} \\right\\rfloor$')
    plt.axhline(1., ls='--', c='red')
    plt.xlabel('$p$')
    plt.legend()
    plt.show()
    
w = interact(compare_edge_counts, n=widgets.IntSlider(description='$n$', min=1, max=200, step=1, value=100, continuous_update=False))

By playing a bit with the previous plot, we can make several observations:
- most of the big differences between the number of edges and the expected value happen when $p$ is small;
- we rarely observe a ratio greater than $1.1$ and smaller than $0.9$;
- when $p$ is not small (typically $p \geq 0.3$), we rarely observe a ratio greater than $1.05$ and smaller than $0.95$.

From these experiments, the variant drawing a graph in $G_{n,p}$ which is simpler than the other one seems to be a rather good approximation of drawing a graph with $\left\lfloor p \binom{n}{2} \right\rfloor$ edges.

### Degree distribution
We are now interested in the **degree distribution** of graphs in $G_{n,p}$. We know that the theoretical distribution of degrees in a graph in $G_{n,p}$ is a binomial law with parameter $n-1$ and $p$.
We first plot the result for *one* graph generated with the `er_np` function, and we plot the corresponding binomial law.

In [ ]:
# degree distribution
n = 1000
p = .5
def degree_distr_er_np(n, p):
    graph_tmp = er_np(n, p)
    got = [0 for k in range(n)]
    for x in graph_tmp.vertex_degrees().values():
        got[x] += 1
    for k in range(n):
        got[k] /= n
    plt.plot(got, label='Measured')
    expected = [binomial_coef(n-1, k) * math.pow(p, k) * math.pow(1-p, n-1-k) for k in range(n)]
    plt.plot(expected, 'r--', label='$\\mathcal{B}(n-1,p)$')
    plt.xlim(n*p - 0.5*n*p*(1-p), n*p + 0.5*n*p*(1-p))
    plt.xlabel('Degree')
    plt.legend()
    plt.show()
w = interact(degree_distr_er_np, n=widgets.IntSlider(description='$n$',min=100,max=1000,step=10,value=700,continuous_update=False)
                             , p=widgets.FloatSlider(description='$p$',min=0.1,max=0.9,step=0.01,value=0.5,continuous_update=False))

We can see that the degree distribution in graphs of $G_{n,p}$ *roughly* follows a binomial law. In particular, the values for which the probability are very close to 0 are the same, and when the probability is non-zero then it has very roughly the same shape. 

In order to smooth a little bit this curve, we make another plot where we take an average degree distribution over a small number of graphs.

**Remark:** in nearly all the following experiments, we take an average over a small number of graphs (less than 20) in order to smooth the results. Since this number is small compared to the total number of possible graphs, it gives us a trend of what happens for most graph, as it is *very* unlikely that we observe a case where there have been a "mean effect" that flaws entirely the result of the experiments. 

In [ ]:
nb = 20

def average_edge_distr_er_np(n, p):
    got = [0 for k in range(n)]
    for i in range(nb):
        graph_tmp = er_np(n, p)
        for x in graph_tmp.vertex_degrees().values():
            got[x] += 1
    for k in range(n):
        got[k] /= n * nb
    plt.plot(got, label='Average measured')
    expected = [binomial_coef(n-1, k) * math.pow(p, k) * math.pow(1-p, n-1-k) for k in range(n)]
    plt.plot(expected, 'r--', label='$\\mathcal{B}(n-1,p)$')
    plt.xlim(n*p - 0.3*n*p*(1-p), n*p + 0.3*n*p*(1-p))
    plt.xlabel('Degree')
    plt.legend()
    plt.show()
w = interact(average_edge_distr_er_np, n=widgets.IntSlider(description='$n$',min=100,max=1000,step=10,value=700,continuous_update=False)
                             , p=widgets.FloatSlider(description='$p$',min=0.1,max=0.9,step=0.01,value=0.5,continuous_update=False))

It is very clear on this new plot that the degree distribution of graphs drawn from $G_{n,p}$ is close to a  binomial law with parameter $n-1$ and $p$. 

### Connected components
We are now interested in the **connected components** of graphs in $G_{n,p}$. We implement a function `biggest_comp_size` that returns the size of the biggest component of a given graph, and `two_biggest_comp_size` that also returns the size of the second biggest component.

In [ ]:
def biggest_comp_size(G):
    comps = G.connected_components()
    # First determine the biggest component #
    comps_size = {}
    for u in comps.values():
        comps_size[u] = 0
    for v,u in comps.items():
        comps_size[u] += 1
    biggest = tuple(comps.values())[0]
    max_size = comps_size[biggest]
    for u in comps.values():
        if comps_size[u] > max_size:
            max_size = comps_size[u]
            biggest = u
    return max_size
def two_biggest_comp_size(G):
    comps = G.connected_components()
    # First determine the biggest component #
    comps_size = {}
    for u in comps.values():
        comps_size[u] = 0
    for v,u in comps.items():
        comps_size[u] += 1
    biggest = tuple(comps.values())[0]
    max_size = comps_size[biggest]
    for u in comps.values():
        if comps_size[u] > max_size:
            max_size = comps_size[u]
            biggest = u
    sec_size = 0
    for u in comps.values():
        if u == biggest:
            continue
        else:
            if comps_size[u] > sec_size:
                sec_size = comps_size[u]
    return max_size, sec_size

In the next plot, we fix some $n$ and see how the size of the biggest component, of the second biggest component, and the number of isolated vertices (i.e. vertices with degree 0) evolve with $p$.

In [ ]:
nb_average = 10

def connected_components_er_np(n):
    end_point = math.ceil(math.log(n,2)+1)
    step = 20*end_point

    bcomp_size = [0 for i in range(step)]
    sec_size = [0 for i in range(step)]
    isolated_size = [0 for i in range(step)]
    x = [0 for i in range(step)]
    for t in range(nb_average):
        for i in range(step):
            p = end_point * i / (step * n) 
            x[i] = n * p
            G = er_np(n, p)
            bcomp, sec = two_biggest_comp_size(G)
            bcomp_size[i] += bcomp
            sec_size[i] += sec
            isolated_size[i] += len(G.find_isolated_vertices())
    bcomp_size = [x / nb_average for x in bcomp_size]
    sec_size = [x / nb_average for x in sec_size]
    isolated_size = [x / nb_average for x in isolated_size]


    plt.plot(x, bcomp_size, label='Biggest component')
    plt.plot(x, sec_size, label='Second biggest component')
    plt.plot(x, isolated_size, label='Isolated vertices')

    #plt.axhline(math.pow(n, 2/3), ls = '--', c='red')
    #plt.text(0., math.pow(n, 2/3)+0.01 * n, '$n^{2/3}$')

    #plt.axvline(1, ls = '--', c = 'red')
    plt.axhline(math.log(n,2), ls='--', c='red')
    plt.text(math.log(n,2)-0.1*math.log(n,2), math.log(n,2)+0.01 * n, '$\log\ n$')

    plt.axvline(math.log(n, 2), ls = '--', c = 'red')
    plt.text(math.log(n,2) - 0.03 * end_point ,n/2,'$\log\ n$',rotation=90)

    plt.legend()
    plt.xlabel('$np$')
    plt.show()
    
w = interact(connected_components_er_np, n=widgets.IntSlider(description='$n$',min=10,max=200,value=100,continuous_update=False))

By changing the value of $n$, we can remark an important fact: when $np$ is greater than $\log n$, the size of the biggest component is almost equal to $n$, i.e. the graph is almost surely connected (the difference is less than 1 when we take the average). We cannot draw a lot of other conclusions from this graph since we need to draw the values as a function of $n$ and here we fixed $n$.

In the next plot, we fix some constant $c>0$ and we draw the size of the biggest component and when it is interesting the size of the second biggest component as a function of $n$, the probability parameter for the random graph generation being $$p=\frac{c}{n}.$$

In [ ]:
# p = c/n with different values of c and let n go big
nb_average = 20

def connected_components_er_np_bis(c, n_max):
    # Determine minimum n such that c/n < 1
    n_min = math.ceil(c)
    bcomp_size = [0 for i in range(n_min,n_max+1)]
    if c > 1.:
        sec_size = [0 for i in range(n_min,n_max+1)]
    x = [i for i in range(n_min,n_max+1)]
    for t in range(nb_average):
        for n in range(n_min,n_max+1):
            G = er_np(n, c/n)
            if c > 1.:
                bcomp, sec = two_biggest_comp_size(G)
                bcomp_size[n-n_min] += bcomp
                sec_size[n-n_min] += sec
            else:
                bcomp = biggest_comp_size(G)
                bcomp_size[n-n_min] += bcomp
    bcomp_size = [x / nb_average for x in bcomp_size]
    if c > 1.:
        sec_size = [x / nb_average for x in sec_size]
    
    plt.plot(x, bcomp_size, label='Biggest component')
    if c > 1.:
        plt.plot(x, sec_size, label='Second biggest')
    
    # Draw hypothesis according to constant c
    if c == 1.:
        hyp = [math.pow(n,2/3) for n in range(n_min, n_max+1)]
        plt.plot(x, hyp, 'r--', label='$n^{2/3}$')
    elif c<1.:
        # Doing a logarithm fit for the biggest component when c>1
        log_fit = np.polyfit(np.log(x), bcomp_size, 1)
        hyp = [log_fit[0]*math.log(n)+log_fit[1] for n in range(n_min, n_max+1)]
        op = ''
        if (log_fit[1] > 0):
            op = '+'
        plt.plot(x, hyp, 'r--', label='$'+str(round(log_fit[0],2))+'\ \log\ x'+op+str(round(log_fit[1],2))+'$')
    else:
        # Doing a logarithm fit for the biggest component when c>1
        log_fit = np.polyfit(np.log(x), sec_size, 1)
        hyp = [log_fit[0]*math.log(n)+log_fit[1] for n in range(n_min, n_max+1)]
        op = ''
        if (log_fit[1] > 0):
            op = '+'
        plt.plot(x, hyp, 'r--', label='$'+str(round(log_fit[0],2))+'\ \log\ x'+op+str(round(log_fit[1],2))+'$')
        lin_fit = np.polyfit(x, bcomp_size, 1)
        hyp2 = [lin_fit[0] * n + lin_fit[1] for n in range(n_min, n_max+1)]
        op = ''
        if (lin_fit[1] > 0):
            op = '+'
        plt.plot(x, hyp2, 'g--', label='$'+str(round(lin_fit[0],2))+'x'+op+str(round(lin_fit[1],2))+'$')
    
    plt.legend()
    plt.xlabel('$n$')
    plt.show()
    
w = interact(connected_components_er_np_bis, n_max=widgets.IntSlider(description='$n_\\text{max}$',min=10,max=200,value=130,step=5,continuous_update=False)
                                           , c=widgets.FloatSlider(description='$c$',min=0.1,max=2.,value=1.,step=0.05,continuous_update=False))

There are three cases that we observe in this plot:
- when $c=1$: the biggest component has a size which is close to the function $n^{2/3}$;
- when $c<1$: the biggest component has a size which is logarithmic as a function of $n$;
- when $c>1$: the biggest component has a size which is linear (i.e. there is a positive fraction of the nodes in the component) and the second biggest component has logarithmic size.

These results corresponds to the well-known characteristics of ER graphs proved by Erdös and Rényi in 1960.

## The configuration model of random networks

In [ ]:
#3
def degree_sequence_regular(n, k):
    return [k for i in range(n)]

def degree_sequence_lognormal(n, mu, sigma):
    return [max(0, math.floor(np.random.lognormal(mu, sigma))) for i in range(n)]

In [ ]:
def configure_sequence(seq):
    non_zero_idx = [ i for i in range(len(seq))]
    
    G = {}
    for i in range(len(seq)):
        G[i] = []
    
    while len(non_zero_idx) > 0:
        id1 = non_zero_idx[random.randrange(len(non_zero_idx))] #get first stubs
        has_removed_1 = False
        if seq[id1] > 1:
            seq[id1] -= 1
        elif seq[id1] == 1:
            seq[id1] = 0
            has_removed_1 = True
            non_zero_idx.remove(id1)
        else:
            has_removed_1 = True
            non_zero_idx.remove(id1)
        
        if len(non_zero_idx) == 0:
            continue
        
        id2 = non_zero_idx[random.randrange(len(non_zero_idx))] #get 2nd stubs
        if seq[id2] > 1:
            seq[id2] -= 1
        elif seq[id2] == 1:
            seq[id2] = 0
            non_zero_idx.remove(id2)
        elif not has_removed_1 and len(non_zero_idx) == 1:
            break
        else:
            if has_removed_1:
                non_zero_idx.append(id1)
            seq[id1] += 1
            continue
            
        G[id1].append(id2)
        G[id2].append(id1)
        
    return Graph(G)

In [ ]:
n_min = 10
n_max = 500
step = 5
k = 20

nb_sample = (n_max - n_min) // step
irregular = [ 0 for i in range(nb_sample)]
x = [ 0 for i in range(nb_sample)]
for i in range(nb_sample):
    x[i] = n_min + i * step
    irregular[i] = configure_sequence(degree_sequence_regular(x[i], k)).irregular_edge_count()
    
plt.plot(x, irregular)
plt.show()

In [ ]:
n_min = 10
n_max = 500
step = 2
mu = 2
sigma = 1

nb_sample = (n_max - n_min) // step
irregular = [ 0 for i in range(nb_sample)]
x = [ 0 for i in range(nb_sample)]
for i in range(nb_sample):
    x[i] = n_min + i * step
    irregular[i] = configure_sequence(degree_sequence_lognormal(x[i], mu, sigma)).irregular_edge_count()
    
plt.plot(x, irregular)
plt.show()